In [ ]:
!pip install keras-tcn

In [ ]:
from tcn import TCN
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy import stats
import warnings
from google.colab import files
warnings.filterwarnings("ignore")
import keras

In [ ]:
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/prediction/tcn/CNN - ET0/'
RUTA2 = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/data/'

import importlib.util

def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module
# Equivalencia a from CNN import Ensemble_estruturado as es
es = module_from_file("es",RUTA+"CNN/Ensemble_estruturado.py")

# Equivalencia a from CNN import Ensemble_estruturado as es
basic = module_from_file("basic",RUTA+"CNN/basic.py")
eCNN = module_from_file("eCNN",RUTA+"CNN/Ensemble_CNN.py")
tcnn = module_from_file("tcnn",RUTA+"CNN/tcn-prediction.py")

In [ ]:
df1 = pd.read_csv(RUTA2+'Eto-Rinconada.csv', sep=',')

In [ ]:
len(df1[:-365])

2002

In [ ]:
def get_search_dataset(i):
    # df1 = pd.read_csv('https://raw.githubusercontent.com/PatriciaLucas/Evapotranspiracao/master/ETo_setelagoas.csv', sep=',')
    # df1 = pd.read_csv(RUTA2+'Eto-Pudahuel.csv', sep=',')
    df1 = pd.read_csv(RUTA2+'Eto-Rinconada.csv', sep=',')
    # df1 = pd.read_csv(RUTA2+'agrometeorologia-20220803074234.csv', sep=',')
    series = df1.iloc[:,1]
    # train = series[:6939]
    # test = series[8035:] #ano de teste: 2015
    train = series[(-365-i):-365]
    test = series[-365:]
    return train, test
    
def load_config(model):
    model.models[0] = {'pilhas': 2,'filters':2,'dropout':0.00438761,'norm':0,'lags':83,'num_conv':4,'kernel_size':5,'tipo':2}
    model.models[1] = {'pilhas': 2,'filters':2,'dropout':0.48961674,'norm':0,'lags':11,'num_conv':1,'kernel_size':11,'tipo':2}
    model.models[2] = {'pilhas': 1,'filters':1,'dropout':0.27417436,'norm':1,'lags':27,'num_conv':3,'kernel_size':5,'tipo':2}
    model.models[3] = {'pilhas': 1,'filters':0,'dropout':0.26143849,'norm':0,'lags':83,'num_conv':5,'kernel_size':5,'tipo':2}
    model.models[4] = {'pilhas': 2,'filters':2,'dropout':0.17972795,'norm':1,'lags':13,'num_conv':2,'kernel_size':2,'tipo':2}
    model.models[5] = {'pilhas': 1,'filters':1,'dropout':0.13627555,'norm':0,'lags':5,'num_conv':2,'kernel_size':3,'tipo':2}
    return model.models

In [ ]:
TYPE_MODEL = 'CNN3'
num = 6
savename = RUTA+'E'+TYPE_MODEL+'_MODELS/Metrics/model_'

In [ ]:
for j in range(840,2000,30):
  train, test = get_search_dataset(j)
  eCNN = module_from_file("eCNN",RUTA+"CNN/Ensemble_CNN.py")
  model = eCNN.Ensemble_CNN(type_models = TYPE_MODEL, number = num) # Creating the ensemble with 6 CNNx models
  model.generate_models()                                           # Generates CNN models
  model.models = load_config(model)
  try:
    model.fit(data = train.values)
    # Point Forecast
    yhat, y_test,_ = model.point_forecast(data = test.values, forecast_horizon=7)
    rmse, mae, mape, r2 = model.metrics(yhat = yhat, y_test=y_test, forecast_horizon=7)
    model_metrics = pd.DataFrame({'RMSE':rmse,'MAE':mae,'MAPE':mape,'R2':r2})
    model_metrics.to_csv(savename+str(j)+'days_back_training'+'-Metrics.csv')
  except:
    print('error de datos')
    continue

error de datos
error de datos
